In [1]:
import os, json, pandas as pd, ast

In [2]:
path = "C:/Users/trive/Desktop/ANC/Project/decentraland/sample/1710779461"
wearables_path = "C:/Users/trive/Desktop/ANC/Project/decentraland/decentraland-wearables"
transactions_path = "C:/Users/trive/Desktop/ANC/Project/decentraland/decentraland"

In [3]:
# Function to parse Decentraland peer data
def parse_decentraland_peers(path):
    records = []

    with open(path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]

    for i in range(0, len(lines), 2):
        url = lines[i]
        response = lines[i + 1] if i + 1 < len(lines) else "{}"
        if "'ok': True" in response:
            response = ast.literal_eval(response)
            for p in response.get("peers", []):
                # don't know if source_peer can be useful later
                p['source_peer'] = url
                records.append(p)
    
    return pd.DataFrame(records)        


In [4]:
# Function to parse Decentraland asset events
def parse_asset_events(path):
    records = []

    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        data = ast.literal_eval(line)

    events = data.get("asset_events", [])
    for e in events:
        nft = e.get("nft", {}) or {}
        payment = e.get("payment", {}) or {}

        record = {
            # Basic event info
            "event_type": e.get("event_type"),
            "closing_date": e.get("closing_date"),
            "seller": e.get("seller"),
            "buyer": e.get("buyer"),
            "transaction": e.get("transaction"),
            "symbol": payment.get("symbol"),
            "price": float(payment.get("quantity", 0)) / (10 ** payment.get("decimals", 18)),
            "nft_id": nft.get("identifier"),
            "nft_name": nft.get("name"),
            "nft_collection": nft.get("collection"),
            "contract": nft.get("contract"),
            "timestamp": e.get("event_timestamp"),
            "nft_image_url": nft.get("image_url"), ## ???
            "metadata_url": nft.get("metadata_url") ## ???
        }

        records.append(record)

    if not records:
        raise ValueError("No valid asset_events found in file.")

    return pd.DataFrame(records)

In [5]:
sample = parse_decentraland_peers(path)
sample.head()  

,id,address,lastPing,parcel,position,source_peer
0,0x08d96844be2b7b05c45d0d81fb6dce03fddb0b66,0x08d96844be2b7b05c45d0d81fb6dce03fddb0b66,1710779462540,"[-27, 56]","[-416.0089111328125, 1.679999828338623, 906.51...",https://peer-eu1.decentraland.org/comms/peers
1,0x0a3dff6b6786e25d08168fdba2c41b550014219d,0x0a3dff6b6786e25d08168fdba2c41b550014219d,1710779461894,"[-29, 55]","[-463, 0.800000011920929, 881]",https://peer-eu1.decentraland.org/comms/peers
2,0xbe3da1d7d3eb69d331435e66adbe2a10c84ebae8,0xbe3da1d7d3eb69d331435e66adbe2a10c84ebae8,1710779461839,"[-101, 127]","[-1608.270751953125, 1.679999828338623, 2045.2...",https://peer.decentral.io/comms/peers
3,0x60a43df8f1c7a4156081717f1396cb587efb95c4,0x60a43df8f1c7a4156081717f1396cb587efb95c4,1710779462575,"[-28, 57]","[-442.08245849609375, 1.679999828338623, 925.0...",https://peer.decentral.io/comms/peers
4,0x5642e6fd2a6d393e774d11fd528862f1b5f8c8d3,0x5642e6fd2a6d393e774d11fd528862f1b5f8c8d3,1710779461027,"[147, 60]","[2361.61572265625, 2.151712417602539, 961.4864...",https://peer.decentral.io/comms/peers


In [6]:
transactions = parse_asset_events(transactions_path)
transactions.head()

,event_type,closing_date,seller,buyer,transaction,symbol,price,nft_id,nft_name,nft_collection,contract,timestamp,nft_image_url,metadata_url
0,sale,1521499616,0x0727fc3970cca8a5e57145777133dc551c124beb,0x5c5b6cade3f45fca78aac5a9877f1b73c51300e3,0xdc12740ef3c4f74e8260842db628bfd6c03c2da47741...,MANA,4000.0,1157920892373161954235709850086879078478254667...,Roadside ArtHaus,decentraland,0xf87e31492faf9a91b02ee0deaad50d51d56d5d4d,1521499616,https://api.decentraland.org/v2/parcels/-16/44...,https://api.decentraland.org/v2/contracts/0xf8...
1,sale,1521499509,0x0727fc3970cca8a5e57145777133dc551c124beb,0x5c5b6cade3f45fca78aac5a9877f1b73c51300e3,0xf9b6cd5d81e3e3963fbb167dcd84ec7faee64868ed3b...,MANA,3000.0,1157920892373161954235709850086879078512282904...,"Parcel -7,-55",decentraland,0xf87e31492faf9a91b02ee0deaad50d51d56d5d4d,1521499509,https://api.decentraland.org/v2/parcels/-7/-55...,https://api.decentraland.org/v2/contracts/0xf8...
2,sale,1521499328,0x0727fc3970cca8a5e57145777133dc551c124beb,0x01a35017c1475e9c4daa88771ee5bc9ee427c28f,0xd7041d908bceede64bba4d154e16e49ec519d3dcad3a...,MANA,3000.0,1701411834604692317316873037158841057170,Double Road Plot,decentraland,0xf87e31492faf9a91b02ee0deaad50d51d56d5d4d,1521499328,https://api.decentraland.org/v2/parcels/4/-110...,https://api.decentraland.org/v2/contracts/0xf8...
3,sale,1521499312,0x0727fc3970cca8a5e57145777133dc551c124beb,0x5c5b6cade3f45fca78aac5a9877f1b73c51300e3,0xddd751cbd94e57c04e7eb60ede7f42314e54378f592f...,MANA,2000.0,1157920892373161954235709850086879078471449020...,"LUKE, I AM YOUR FATHER",decentraland,0xf87e31492faf9a91b02ee0deaad50d51d56d5d4d,1521499312,https://api.decentraland.org/v2/parcels/-18/41...,https://api.decentraland.org/v2/contracts/0xf8...
4,sale,1521499060,0x0727fc3970cca8a5e57145777133dc551c124beb,0x00e8c2f1e2a359f7295430ee81a67bbbe46cfe6f,0xc2524d3b165580ca86751f43978ec52c522fa3716a6a...,MANA,3000.0,5104235503814076951950619111476523171808,MAX BUILD HEIGHT = 60 M/190 FT (ALL 7 PARCELS),decentraland,0xf87e31492faf9a91b02ee0deaad50d51d56d5d4d,1521499060,https://api.decentraland.org/v2/parcels/14/-32...,https://api.decentraland.org/v2/contracts/0xf8...


In [7]:
wearables = parse_asset_events(wearables_path)
wearables.head()

,event_type,closing_date,seller,buyer,transaction,symbol,price,nft_id,nft_name,nft_collection,contract,timestamp,nft_image_url,metadata_url
0,sale,1541618795,0xaebb23493e64a06c7fcb884cb1dbeab5fa6c84b5,0x15be42b31b20308e64adc225ed1ad99910ce4157,0xdad5af351b9eba5a9e2cfa847710e559b53c065de2f2...,MANA,155000.0,935,🔥 Estate for FASHION MALL 🔥,decentraland,0x959e104e1a4db6317fa58f8295f586e1a978c297,1541618795,https://api.decentraland.org/v2/estates/935/ma...,https://api.decentraland.org/v2/estates/935
1,sale,1541618211,0xaebb23493e64a06c7fcb884cb1dbeab5fa6c84b5,0x81e4fb0c64bf49f89b57f6648562fc9a791b2e92,0x8f4c20af6e00e14cc35d238442d0c2f6778e89e7e9cd...,MANA,42000.0,937,"Great Estate, Ready to Auction 2",decentraland,0x959e104e1a4db6317fa58f8295f586e1a978c297,1541618211,https://api.decentraland.org/v2/estates/937/ma...,https://api.decentraland.org/v2/estates/937
2,sale,1541614875,0xdfe9e4170953bd73be4a68411aa4707850f16ce3,0x15be42b31b20308e64adc225ed1ad99910ce4157,0xcc43c309dbf89d4103fa9daff6b71ab3573505156670...,MANA,29500.0,906,🏛🏆 LAST OF THE COMUNITY 🏆🏛,decentraland,0x959e104e1a4db6317fa58f8295f586e1a978c297,1541614875,https://api.decentraland.org/v2/estates/906/ma...,https://api.decentraland.org/v2/estates/906
3,sale,1541574048,0xc93a05bbd0e03c2c4a8f0d0165d52ff901ad659c,0xaa73a811a4898ca0e116376be752b3960d554021,0x05cf68c9cd0285ffcb8cea0ca0738ba231f14e415a2e...,MANA,36000.0,284,Road Bridge,decentraland,0x959e104e1a4db6317fa58f8295f586e1a978c297,1541574048,https://api.decentraland.org/v2/estates/284/ma...,https://api.decentraland.org/v2/estates/284
4,sale,1541561884,0xb79deec031aec829ce4cf38e1d53df5b6f50668f,0xabab1e5eae3fda3a461e1ce49f117057a2f106ce,0x735cd205c0bed669720a9751e047f3aaad8e1a8a4e20...,MANA,99999.0,920,Dragon Hills,decentraland,0x959e104e1a4db6317fa58f8295f586e1a978c297,1541561884,https://api.decentraland.org/v2/estates/920/ma...,https://api.decentraland.org/v2/estates/920


## **Recap of possible strategy**

## Defining an Influencer in Decentraland

Since we cannot rely on the same definition of *influencer* used in the real world,  
we define it instead as **a wallet (or user) capable of attracting, connecting, and mobilizing other users inside the virtual world**.

The idea is that the **presence** of someone can influence **how people gather, interact, or move** inside the Decentraland space (and time).

---

### Datasets

1. **Social Presence Layer**  
   Captures *who is online, where, and when* — representing the spatial-temporal distribution of users.

2. **Economic Activity Layer**  
   Records *land transactions* and economic interactions between wallets.

3. **Cultural Dynamics Layer**  
   Covers *wearable NFT trades*, representing aesthetic, symbolic, or cultural influence.

---

### Network Modeling Strategy

Each dataset will be modeled as a **network**:

- **Nodes:** wallets / users  
- **Edges:** interactions or transactions  

We will analyze each network **separately** and then integrate them into a **multilayer network**.

---

### Goal

Identify users who consistently appear **central or impactful across layers** — 
these are our **potential influencers in Decentraland**.

